## Devices Classifier with nPrintML

#### Before execution

```bash
conda create -n nprint python=3.8
conda activate nprint
python -m pip install nprintml

sudo ln -s ../nprint  /usr/local/bin/nprint
```

Then we have to slightly modify the nPrintML code base:

For file: `~/miniconda3/envs/nprint/lib/python3.8/site-packages/nprintml/net/step.py`
Add the folloing python code to L39
```python3
self.group_parser.add_argument(
            '-w', '--wlan',
            action='store_true',
            help="include wlan headers",
        )
```

In [1]:
import pandas as pd
from mac_vendor import *

In [2]:
# generate nprint.npt file
filename = 'wlan_2020_11_05_03'

filter_cmd = f'tshark -r {filename}.pcap -w {filename}_with_src_mac.pcap "wlan.ta"'
!{filter_cmd}

filename = filename + '_with_src_mac'

nprint_cmd = f'../nprint -w -P {filename}.pcap -W {filename}.npt'
!{nprint_cmd}


In [3]:
# Create labels file for nprint ML

nprint_wlan = pd.read_csv(f'{filename}.npt', index_col=0)
top_10_vendors = get_top_vendors(nprint_wlan, 10)

# create labels file
labels_path = f'{filename}-vendor-labels.txt'
labels_map = {}

for mac in nprint_wlan.index:
    if mac == 'None': labels_map[mac] = 'None'
    else:
        vendor = get_vendor(mac)
        if vendor in top_10_vendors: labels_map[mac] = vendor
        else: labels_map[mac] = 'Others'

with open(labels_path, 'w') as f:
    f.write('item,label\n')
    for k, v in labels_map.items():
        f.write(f'{k},{v}\n')


In [4]:
# Run nprintml
nml_cmd = f'nprintml --wlan --pcap-file {filename}.pcap --label-file {labels_path} --aggregator index'
!{nml_cmd}

[warn] nprint expected version for nprintML (1.1.6) does not match version on PATH (1.0.4 at /usr/local/bin/nprint)
step:Net → NetResult(nprint_path=PosixPath('nprintml/run-falcon-1615500625-22534/nprint'))
Loading nPrint: nprintml/run-falcon-1615500625-22534/nprint/wlan_2020_11_05_03_with_src_mac.npt
Loaded 1 nprint
  nPrint shape: (3910, 528)
Loading labels: wlan_2020_11_05_03_with_src_mac-vendor-labels.txt
  number of labels: 91
Attaching labels to nPrints
  labels attached: missing labels for:
    missing labels caused samples to be dropped: 0
step:Label → LabelResult(features=                   radiotap_reversion_0  ...   label
06:7f:2c:c3:10:90                     0  ...  Others
08:02:8e:ca:ae:73                     0  ...  Others
08:02:8e:ca:ae:73                     0  ...  Others
08:02:8e:ca:ae:73                     0  ...  Others
08:ea:40:f9:0e:44                     0  ...  Others
...                                 ...  ...     ...
ff:ff:ff:ff:60:e1                     0  